# Paso a parquet el archivo de contratos

In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
conf = (SparkConf().set("spark.executor.memory", "8g")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ContratosAParque").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/22 10:04:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import substring

esquema = StructType() \
      .add("ctId",StringType(),True) \
      .add("fecIni",StringType(),True) \
      .add("fecFin",StringType(),True) \
      .add("dato",StringType(),True) \
      .add("saldo",IntegerType(),True)

df = spark.read.csv("./datos/fakeContracts01_sort_Fechas.csv", sep=";", header = True, schema = esquema)
print(df.count())
df = df.withColumn("proc", substring("ctId",1,6))
df.write.mode("append").partitionBy("proc").parquet("./datos/fakeContracts01.parquet")

1426246


In [3]:
spark.sparkContext.stop()